In [1]:
import sys
import json
import numpy as np
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [2]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [3]:
import os
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numba as nb
from quetzal.model import stepmodel
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

num cores: 8


assert False

In [4]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

In [5]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
if 'params' in argv.keys():
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())
    
cutoff = var['pathfinder']['cutoff']   

# inputs

In [6]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')
distribution = stepmodel.read_zippedpickles(model_folder + 'distribution')

sm.volumes = distribution.volumes

volumes: 100%|██████████| 1/1 [00:00<00:00, 98.53it/s]


# od_set

In [7]:
v = sm.volumes.set_index(['origin', 'destination']).sum(axis=1)
od_set = set(v.loc[v>0].index)
len(od_set)

67780

In [8]:
cutoff=7200

In [9]:
%load_ext snakeviz
%snakeviz_config -h localhost -p 8080

Snakeviz configured with host localhost and port 8080


# PT pathfinder

In [15]:
sm.step_pt_pathfinder(
    broken_routes=True,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    walk_on_road=True,
    path_analysis=False,
    od_set=od_set,
    cutoff=cutoff,
    verbose=False,
)
ppf = sm.publicpathfinder

start publicpathfinder


route_type {'tram'}: 100%|██████████| 3/3 [00:02<00:00,  1.49it/s]       

path_analysis


In [14]:
%%snakeviz
self = ppf

broken_routes = True
broken_modes = True
mode_column = 'route_type'
route_column = 'route_id'
verbose = False

self.combinations = dict()
# BUILD ALL MODE COMBINATIONS 
mode_combinations = [set()]
modes = set(self.links[mode_column])
for mode in modes:
    mode_combinations += [s.union({mode}) for s in mode_combinations] 
self.combinations[mode_column] = mode_combinations[1:] # remove empty set

# BUILD ROUTE COMBINATIONS | ONLY ONE ROUTE BROKEN IN EACH COMBINATION
broken_route_set = set(self.links[route_column])
self.combinations[route_column] = [{route} for route in broken_route_set]

# BUILD OD SETS
self.build_route_id_sets(first_and_last_only=False)
self.build_route_type_sets(first_and_last_only=False)
self.build_od_sets(split_factor=0.5, verbose=verbose)


# FIND BROKEN ROUTES
self.broken_route_paths = pd.DataFrame()
if broken_routes:
    self.find_broken_combination_paths(column='route_id', cutoff=cutoff, build_shortcuts=False, prune=False, reuse_matrix=True, keep_matrix=True, log=verbose)
    self.broken_route_paths = self.broken_combination_paths
    self.broken_route_paths['pathfinder_session'] = 'route_breaker' 
    self.broken_route_paths['broken_route'] = self.broken_route_paths['broken_' + route_column].apply(
        lambda s: list(s)[0]
    ) # we assume only one route is broken at a time

# FIND BROKEN PATHS
self.broken_mode_paths = pd.DataFrame()
if var.get('pathfinder',{}).get('mode_breaker') or True:
    self.find_broken_combination_paths(column='route_type', cutoff=cutoff, build_shortcuts=False, prune=False, reuse_matrix=True, log=verbose, keep_matrix=True)
    self.broken_mode_paths = self.broken_combination_paths
    self.broken_mode_paths['pathfinder_session'] = 'mode_breaker'
    self.broken_mode_paths['broken_modes'] = self.broken_mode_paths['broken_' + mode_column].apply(set)

self.paths = pd.concat([
    self.best_paths,
    self.broken_mode_paths, 
    self.broken_route_paths, 
    ]
)
self.paths['path'] = [tuple(p) for p in self.paths['path']]
self.paths = self.paths.drop_duplicates(subset=['path'])

route_type {'tram'}: 100%|██████████| 3/3 [00:02<00:00,  1.38it/s]       


 
*** Profile stats marshalled to file 'C:\\Users\\QCHASS~1\\AppData\\Local\\Temp\\tmpw_frynut'.
Embedding SnakeViz in this document...
<function display at 0x0000021CAF0F2280>


In [13]:
ppf.paths.set_index(['origin', 'destination'])['pathfinder_session'].value_counts()

route_breaker    10239
best_path         7052
mode_breaker      1539
Name: pathfinder_session, dtype: int64

In [15]:
self.build_od_sets(split_factor=0.5, verbose=verbose)

# Pruning

### Set broken routes and modes

In [ ]:
broken_bus = []
broken_routes = set(sm.links['route_id'])
print('Broken Routes:', broken_routes)

In [ ]:
mode_combinations = []
if var.get('pathfinder',{}).get('walk_pathfinder'):
    mode_combinations.append(set(sm.links['route_type'].unique()))

for mode in sm.links['route_type'].unique():
    mode_combinations.append({mode})
    
print('Broken Modes Combinaison:', mode_combinations)

In [ ]:
self.combinations = dict()
self.combinations['route_id'] = [{route} for route in broken_routes]
self.combinations['route_type'] = mode_combinations

self.build_route_id_sets(first_and_last_only=False)
self.build_route_type_sets(first_and_last_only=False)
self.build_od_sets(split_factor=0.5, verbose=True)


In [ ]:
from quetzal.engine.pathfinder_utils import *

In [ ]:
def pt(path):
    for i in path:
        if 'link' in i:
            return True
    return False

In [ ]:
sm.pt_los['pt'] = sm.pt_los['path'].apply(pt)
sm.pt_los['pt'].sum()

In [ ]:
link_e = link_edge_array(ppf.links, 300)
footpaths_e = ppf.footpaths[['a', 'b', 'time']].values
ntlegs_e = ppf.ntlegs[['a', 'b', 'time']].values
edges = np.concatenate([link_e, footpaths_e, ntlegs_e])

In [ ]:
#%%snakeviz 
paths_from_edges(edges, od_set={('zone_8', 'zone_36')}, cutoff=np.inf)

In [ ]:
#%%snakeviz
ppf.broken_route_paths = pd.DataFrame()


if var.get('pathfinder',{}).get('route_breaker') or True:
    ppf.find_broken_combination_paths(column='route_id', cutoff=cutoff, build_shortcuts=False, prune=False, reuse_matrix=False)
    ppf.broken_route_paths = ppf.broken_combination_paths
    ppf.broken_route_paths['pathfinder_session'] = 'route_breaker' 
    ppf.broken_route_paths['broken_route'] = ppf.broken_route_paths['broken_route_id'].apply(
        lambda s: list(s)[0]
    )

In [ ]:
#%%snakeviz
ppf.broken_mode_paths = pd.DataFrame()
if var.get('pathfinder',{}).get('mode_breaker') or True:
    ppf.find_broken_combination_paths(column='route_type', cutoff=cutoff, build_shortcuts=False, prune=False, reuse_matrix=True, log=True, keep_matrix=False)
    ppf.broken_mode_paths = ppf.broken_combination_paths
    ppf.broken_mode_paths['pathfinder_session'] = 'mode_breaker'
    ppf.broken_mode_paths['broken_modes'] = ppf.broken_mode_paths['broken_route_type'].apply(set)

In [ ]:
pt_los = pd.concat([
    ppf.best_paths,
    ppf.broken_mode_paths, 
    ppf.broken_route_paths, 
    ]
)
pt_los['path'] = [tuple(p) for p in pt_los['path']]
pt_los = pt_los.drop_duplicates(subset=['path'])

print(len(pt_los))
sm.pt_los = pt_los
pt_los['pt'] = pt_los['path'].apply(pt)
pt_los.set_index(['origin', 'destination'])['pathfinder_session'].value_counts()


In [ ]:
paths_from_edges(edges, od_set={('zone_310', 'zone_196')}, cutoff=np.inf, csgraph=pmatrix, node_index=ppf.node_index)

In [ ]:
#pmatrix[ppf.node_index['link_6067']].data += np.inf
pmatrix[ppf.node_index['link_6067']].data

In [ ]:
pmatrix[ppf.node_index['link_6067']].data = pmatrix[ppf.node_index['link_6067']].data + np.inf
pmatrix[ppf.node_index['link_6067']].indices

In [ ]:
i = [ppf.node_index['link_6067']]
for j in pmatrix[i].indices:
    pmatrix[i,j] = np.inf

In [ ]:
pmatrix[i].data

In [ ]:
print(dir(pmatrix[ppf.node_index['link_6067']]))

In [ ]:
rid_set = set(ppf.links['route_id'])

In [ ]:
mylos = pt_los.loc[(pt_los['broken_route_type'] == frozenset(['bus'])) & (pt_los['pt']== True)]
paths =  mylos['path']
route_types = ppf.links['route_type'].to_dict()
route_type_set = set()
for p in paths : 
    route_type_set = route_type_set.union({route_types.get(n, None) for n in p})
route_type_set

In [ ]:
for route_id in rid_set:
    mylos = pt_los.loc[(pt_los['broken_route_id'] == frozenset([])) & (pt_los['pt']== True)]
    paths =  mylos['path']
    route_types = ppf.links['route_id'].to_dict()
    route_type_set = set()
    for p in paths : 
        route_type_set = route_type_set.union({route_types.get(n, None) for n in p})
    assert route_id not in route_type_set

In [ ]:
mylos

In [ ]:
route_types = ppf.links['route_type'].to_dict()
route_type_set = set()
for p in paths : 
    route_type_set = route_type_set.union({route_types.get(n, None) for n in p})
route_type_set

In [ ]:
pt_los.loc[(pt_los['broken_route_type'] == frozenset(['bus'])) & (pt_los['pt']== True)]['path']

In [ ]:

sm.step_road_pathfinder(method='aon', access_time='time',time_column='time',od_set=od_set,path_analysis=True)

In [ ]:
sm.analysis_pt_los(walk_on_road=True)
sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])

In [ ]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])